In [ ]:
# importing libirary
import warnings
import shap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier as RFC, GradientBoostingClassifier as GBC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,RandomizedSearchCV
from skopt import BayesSearchCV
from bayes_opt import BayesianOptimization
import sklearn.neighbors._base
# before import missingpy, you must run the following codes
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
warnings.filterwarnings('ignore')


In [ ]:
# loading data
# Please change the following path to the file you want to analyze
raw_data = pd.read_csv("somepath/yourfile.csv")


X = raw_data.iloc[:, raw_data.columns != 'unmet ADL needs']
y = raw_data.iloc[:, raw_data.columns == 'unmet ADL needs']
cols = X.columns

# imputing missing data
imputer = MissForest()
imputer.fit(X)
X2 = imputer.transform(X)
X2 = np.round(X2)
X = pd.DataFrame(X2, columns=cols)

# spliting data
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y, test_size=0.3, random_state=11)

for i in [Xtrain, Xtest, ytrain, ytest]:
    i.index = range(i.shape[0])


In [ ]:
# searching the best parameters of logistic regression model
# it will last a few minutes
param_gird = {'C': list(np.linspace(0.001, 1, 50)),
              'solver': ['liblinear', 'sag', 'newton-cg', 'lbfgs']}


LRTC = LR()
GS = GridSearchCV(LRTC, param_gird, cv=10, n_jobs=-1, scoring='roc_auc')
GS.fit(Xtrain, ytrain)
bestParams_LR = GS.best_params_
print(bestParams_LR, GS.best_score_)

# printing performance of logistic regression model with the best parameters
clf = LR(**bestParams_LR)
# using train dataset to train model
clf.fit(Xtrain, ytrain)
# using test dataset to test the performance of model
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)

In [ ]:
# searching the best parameters of gradient boosting model
# it will last a few minutes
def gbc_cv(n_estimators, max_depth, max_features, min_samples_leaf, min_samples_split, learning_rate):
    gbc = GBC(n_estimators=round(n_estimators), max_depth=round(max_depth),
              max_features=max_features, min_samples_leaf=round(min_samples_leaf),
              min_samples_split=round(min_samples_split), learning_rate=learning_rate)
    roc_aucs = cross_val_score(gbc, Xtrain, ytrain, cv=10, scoring='roc_auc', n_jobs=-1)
    return np.mean(roc_aucs)

gbc_op = BayesianOptimization(
    gbc_cv,
    {'n_estimators': (1, 200),
     'max_depth': (1, 20),
     'max_features': (0.1, 1.0),
     'min_samples_leaf': (1, 20),
     'min_samples_split': (2, 20),
     'learning_rate': (0.001, 1)},
    random_state=2023
)

gbc_op.maximize(init_points=10, n_iter=50)
bestParams_GBC = gbc_op.max['params']
# Rounding a floating-point value
for k, v in bestParams_GBC.items():
    if k not in ['max_features', 'learning_rate']:
        bestParams_GBC[k] = round(v)
print(bestParams_GBC, gbc_op.max['target'])

# printing performance of gradient boosting model with the best parameters
clf = GBC(**bestParams_GBC)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)

In [ ]:
# searching the best parameters of naïve bayes model
param_gird = {'var_smoothing': list(np.linspace(0.01, 1, 100))}
nb = NB()
GS = GridSearchCV(nb, param_gird, cv=10, n_jobs=-1, scoring='roc_auc')
GS.fit(Xtrain, ytrain)
bestParams_NB = GS.best_params_
print(bestParams_NB, GS.best_score_)

# printing performance of naïve bayes model
clf = NB(**bestParams_NB)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)

In [ ]:
# searching the best parameters of decision tree model
# it will last a few minutes
param_gird = {'max_depth': np.arange(1, 20, 2),
              'max_features': np.arange(1, 20, 2),
              'min_samples_leaf': np.arange(2, 20, 2),
              'min_samples_split': np.arange(4, 20, 2)}
dt = DT()
GS = RandomizedSearchCV(dt, param_gird, cv=10, n_jobs=-1, scoring='roc_auc')
GS.fit(Xtrain, ytrain)
bestParams_DT = GS.best_params_
print(bestParams_DT, GS.best_score_)

# printing performance of decision tree model
clf = DT(**bestParams_DT)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)

In [ ]:
# searching the best parameters of KNN model
param_gird = {'n_neighbors': list(np.arange(1, 30))}
knn = KNN()
GS = GridSearchCV(knn, param_gird, cv=10, n_jobs=-1, scoring='roc_auc')
GS.fit(Xtrain, ytrain)
bestParams_KNN = GS.best_params_
print(bestParams_KNN, GS.best_score_)

# printing performance of KNN model
clf = KNN(**bestParams_KNN)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)


In [ ]:
# using BayesianOptimization to search the best parameters of random forest model
def rfc_cv(n_estimators, min_samples_split, max_features, max_depth):
    clf = RFC(n_estimators=round(n_estimators),  min_samples_split=round(min_samples_split),
              max_features=max_features,
              max_depth=round(max_depth), random_state=2023)
    
    # 10-fold cross validation
    roc_aucs = cross_val_score(clf, Xtrain, ytrain, cv=10, scoring='roc_auc', n_jobs=-1)
    return np.mean(roc_aucs)

rfc_op = BayesianOptimization(
    rfc_cv,
    {'n_estimators': (10, 200),
     'min_samples_split': (5, 25),
     'max_features': (0.1, 0.999),
     'max_depth': (3, 20)},
    random_state=2023
)

rfc_op.maximize(init_points=10, n_iter=50)
bestParams_RFC = rfc_op.max['params']
# Rounding a floating-point value
for k, v in bestParams_RFC.items():
    if k != 'max_features':
        bestParams_RFC[k] = round(v)
print(bestParams_RFC, rfc_op.max['target'])

# printing performance of random forest model with the best parameters
clf = RFC(random_state=2023, **bestParams_RFC)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)
yproba = clf.predict_proba(Xtest)
print('Accuracy:', clf.score(Xtest, ytest))
print('Precision:', metrics.precision_score(ytest, ypred))
print('Recall:', metrics.recall_score(ytest, ypred))
print('F1 score:', metrics.f1_score(ytest, ypred))
roc_auc = metrics.roc_auc_score(ytest, yproba[:, 1])
print('AUROC:', roc_auc)

In [ ]:
# ploting ROC curve
plt.figure(figsize=(20, 16))
model_rfc = RFC(random_state=2023, **bestParams_RFC)
model_rfc.fit(Xtrain, ytrain)
y_pred_rfc = model_rfc.predict_proba(Xtest)[:, 1]
fpr_rfc, tpr_rfc, _ = roc_curve(ytest, y_pred_rfc)
auc_rfc = round(metrics.roc_auc_score(ytest, y_pred_rfc), 3)
plt.plot(fpr_rfc, tpr_rfc, label="Random Forest, AUROC="+str(auc_rfc))
plt.legend()

model_gbc = GBC(**bestParams_GBC)
model_gbc.fit(Xtrain, ytrain)
y_pred_gbc = model_gbc.predict_proba(Xtest)[:, 1]
fpr_gbc, tpr_gbc, _gbc = roc_curve(ytest, y_pred_gbc)
auc_gbc = round(metrics.roc_auc_score(ytest, y_pred_gbc), 3)
plt.plot(fpr_gbc, tpr_gbc, label="Gradient Boosting, AUROC="+str(auc_gbc))

model_lr = LR(**bestParams_LR)
model_lr.fit(Xtrain, ytrain)
y_pred_lr = model_lr.predict_proba(Xtest)[:, 1]
fpr_lr, tpr_lr, _ = roc_curve(ytest, y_pred_lr)
auc_lr = round(metrics.roc_auc_score(ytest, y_pred_lr), 3)
plt.plot(fpr_lr, tpr_lr, label="Logistic Regress, AUROC="+str(auc_lr))
plt.legend()

model_nb = NB(**bestParams_NB)
model_nb.fit(Xtrain, ytrain)
y_pred_nb = model_nb.predict_proba(Xtest)[:, 1]
fpr_nb, tpr_nb, _nb = roc_curve(ytest, y_pred_nb)
auc_nb = round(metrics.roc_auc_score(ytest, y_pred_nb), 3)
plt.plot(fpr_nb, tpr_nb, label="Naïve Bayes, AUROC="+str(auc_nb))

model_dt = DT(**bestParams_DT)
model_dt.fit(Xtrain, ytrain)
y_pred_dt = model_dt.predict_proba(Xtest)[:, 1]
fpr_dt, tpr_dt, _dt = roc_curve(ytest, y_pred_dt)
auc_dt = round(metrics.roc_auc_score(ytest, y_pred_dt), 3)
plt.plot(fpr_dt, tpr_dt, label="Decision Tree, AUROC="+str(auc_dt))

model_knn = KNN(**bestParams_KNN)
model_knn.fit(Xtrain, ytrain)
y_pred_knn = model_knn.predict_proba(Xtest)[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(ytest, y_pred_knn)
auc_knn = round(metrics.roc_auc_score(ytest, y_pred_knn), 3)
plt.plot(fpr_knn, tpr_knn, label="K_Nearest Neighbor, AUROC="+str(auc_knn))
plt.legend()

lw = 2
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.title('ROC Curve', fontsize=20)

plt.rcParams.update({'font.size': 20})
plt.legend(loc='lower right')
plt.savefig('./roc_curve.png', dpi=300, bbox_inches='tight',
            transparent=True, facecolor='white')

In [ ]:
# ploting SHAP
clf = RFC(random_state=2023, **bestParams_RFC)
shap.initjs()
clf.fit(X, y.values)
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)[1]
shap.summary_plot(shap_values, X, plot_type="bar")
shap.summary_plot(shap_values, X)
